# Gold Price VS Import Values and Export Values ##

We are interested in determing the relationship between gold price versus import values and gold price versus export values. To find out, we obtain monthly gold price from Internet, monthly import and export value for different countries from IMF data library. 

## Extract import values and Export values

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline
import numpy as np
import statsmodels.formula.api as smf

Define functions to split values for different countries

In [ ]:
# get import values for select country
def selectImportCountry(country):
    selector = import_data[['Country Name']].values == country
    temp = import_data[selector]
    temp = temp[temp['Time Period'].astype('string').str.contains('M')]
    temp['Time Period'] = temp['Time Period'].astype('string').str.replace('M','-')
    temp['Time Period'] = pd.to_datetime(temp['Time Period'])
    temp.sort_values(['Time Period'], ascending = 1)
    return temp.drop(['Indicator Code','Country Code'], axis = 1)

In [ ]:
# get export vlues for select country
def selectExportCountry(country):
    selector = export_data[['Country Name']].values == country
    temp = export_data[selector]
    temp = temp[temp['Time Period'].astype('string').str.contains('M')]
    temp['Time Period'] = temp['Time Period'].astype('string').str.replace('M','-')
    temp['Time Period'] = pd.to_datetime(temp['Time Period'])
    temp.sort_values(['Time Period'], ascending = 1)
    return temp.drop(['Indicator Code','Country Code'], axis = 1)

In [ ]:
# calculate the correlation between gold price and import or export values
def calCorrelation(dataFrame):
    corr = dataFrame['gold price'].corr(dataFrame['Value'])
    print corr

In [ ]:
# normalized data
def normalizeDate(dataframe):
    dataframe['normalized gold price'] = \
    (dataframe['gold price']- dataframe['gold price'].mean())/(dataframe['gold price'].std()/np.sqrt(dataframe['Time Period'].size))
    dataframe['normalized value'] = \
    (dataframe['Value'] - dataframe['Value'].mean())/(dataframe['Value'].std()/np.sqrt(dataframe['Time Period'].size))                                                             

In [ ]:
# calculate the correlation after normalization
def calNormCorrelation(dataFrame):
    normalCorr = dataFrame['normalized gold price'].corr(dataFrame['normalized value'])
    print normalCorr

In [ ]:
# plot time vs gold price and import or export values
def plotTimeSeriesData(dataFrame, isImport):
    dataFrame.plot(x = ['Time Period'], y = ['normalized gold price','normalized value'])
    if(isImport):
        plt.title('Gold Price VS ' + dataFrame['Country Name'][0] + ' import values')
    else:
        plt.title('Gold Price VS ' + dataFrame['Country Name'][0] + ' export values')

In [ ]:
# summarize results
def summaryResult(dataFrame):
    x = dataFrame['normalized gold price']
    y = dataFrame['normalized value']
    results = smf.OLS(y,x).fit()
    print dataFrame['Country Name'][0]
    print results.summary()
    print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n \n \n"

Fetch date from pre-processed dataset

In [ ]:
import_data = pd.read_csv('import_value.csv').drop('Unnamed: 0',1).drop('Status',1).drop('Unnamed: 7',1)
export_data = pd.read_csv('export_value.csv').drop('Unnamed: 0',1).drop('Status',1).drop('Unnamed: 7',1)
gold_price = pd.read_csv('gold price.csv')
gold_price.columns=['Time Period','gold price']
gold_price['Time Period'] = pd.to_datetime(gold_price['Time Period'])

In [ ]:
# final all unique countries in both import data and export data
country_name_1 = import_data['Country Name'].unique()
country_name_2 = export_data['Country Name'].unique()
country_name = set(country_name_1).intersection(country_name_2)
country_name =list(country_name)


In [ ]:
# generate import and exprot countries lists
country_name_import = []
country_name_export = []
N_countries = len(country_name)
for i in range(0,N_countries):
    country_name_import.append(country_name[i] + "" + "_import")
    country_name_export.append(country_name[i] + "" + "_export")

In [ ]:
# extract import and export values for each country
for i in range(N_countries):
    country_name_import[i] = selectImportCountry(country_name[i]).sort_values(['Time Period'], ascending = 1)
    country_name_export[i] = selectExportCountry(country_name[i]).sort_values(['Time Period'], ascending = 1)

In [ ]:
# delete empty dataframe in import data
temp_import_list = []
for i in range(N_countries):
    temp_import_df = country_name_import[i]
    if(temp_import_df.size != 0):
        temp_import_list.append(temp_import_df)
country_name_import = temp_import_list

In [ ]:
# delete empty dataframe in import data
temp_export_list = []
for i in range(N_countries):
    temp_export_df = country_name_export[i]
    if(temp_export_df.size != 0):
        temp_export_list.append(temp_export_df)
country_name_export = temp_export_list

In [ ]:
# create two new list to contain gold price versus import and export values for each countries
gold_country_import = []
gold_country_export = []
N_import = len(country_name_import)
N_export = len(country_name_export)
for i in range(N_import):
    gold_country_import.append("gold_" + country_name[i] + "_import")
for i in range(N_export):
    gold_country_export.append("gold_" + country_name[i] + "_export")

In [ ]:
# concatenate gold price and import, export values
for i in range(N_import):
    gold_country_import[i] = pd.merge(gold_price, country_name_import[i], on='Time Period')
for i in range(N_export):
    gold_country_export[i] = pd.merge(gold_price, country_name_export[i], on='Time Period')

In [ ]:
# normalize the data 
for i in range(N_import):
    normalizeDate(gold_country_import[i])
for i in range(N_export):
    normalizeDate(gold_country_export[i])

In [ ]:
# calculate and print out correlation after normalization
for i in range(N_import):
    print "======================================="
    print country_name[i]
    print "Gold Price VS Import Value Correlation"
    calNormCorrelation(gold_country_import[i])
    print gold_country_export[i]['Country Name'][0]
    print "Gold Price VS Export Value Correlation"
    calNormCorrelation(gold_country_export[i])
    print "=======================================\n"

In [ ]:
# plot time series vs gold price and import, export values
for i in range(N_import):
    plotTimeSeriesData(gold_country_import[i], True)

In [ ]:
for i in range(N_export):
    plotTimeSeriesData(gold_country_export[i], False)

In [ ]:
# Summarize the results
for i in range(N_import):
    summaryResult(gold_country_import[i])

In [ ]:
# Summarize the results
for i in range(N_export):
    summaryResult(gold_country_export[i])